In [3]:
import pandas as pd
import numpy as np

# ----------------------------------------------------
# 1. LOAD DATA
# ----------------------------------------------------
geo = pd.read_csv("geolocation.csv")
trucks_mg = pd.read_csv("trucks_mg.csv")

# clean column names
geo.columns = geo.columns.str.strip().str.lower()
trucks_mg.columns = trucks_mg.columns.str.strip().str.lower()

# ----------------------------------------------------
# 2. CALCULATE RISKFACTOR (Safety Score)
# ----------------------------------------------------
# abnormal events = event_ind = 1
risk_df = geo.groupby("driverid")["event_ind"].mean().reset_index()
risk_df.rename(columns={"event_ind": "riskfactor"}, inplace=True)

# normalize riskfactor (min=0, max=1)
risk_df["riskfactor"] = (risk_df["riskfactor"] - risk_df["riskfactor"].min()) / \
                         (risk_df["riskfactor"].max() - risk_df["riskfactor"].min())

# round for consistency with screenshot
risk_df["riskfactor"] = risk_df["riskfactor"].round(3)

# ----------------------------------------------------
# 3. TOTAL ABNORMAL EVENTS PER DRIVER
# ----------------------------------------------------
events_df = geo.groupby("driverid")["event_ind"].sum().reset_index()
events_df.rename(columns={"event_ind": "total_events"}, inplace=True)

# ----------------------------------------------------
# 4. CALCULATE AVERAGE MPG FOR EACH DRIVER
# ----------------------------------------------------
# first compute mpg per truck
trucks_mg["mpg"] = trucks_mg["miles"] / trucks_mg["gas"]

# average mpg per driver
mpg_df = trucks_mg.groupby("driverid")["mpg"].mean().reset_index()
mpg_df.rename(columns={"mpg": "avgmpg"}, inplace=True)

# ----------------------------------------------------
# 5. MERGE ALL THREE TABLES
# ----------------------------------------------------
final_df = risk_df.merge(events_df, on="driverid", how="left")
final_df = final_df.merge(mpg_df, on="driverid", how="left")

# ----------------------------------------------------
# 6. COMPUTE PERFORMANCE SCORE
# ----------------------------------------------------
# Formula (matches screenshot behavior):
# Higher mpg = better
# Lower riskfactor = better
# Fewer events = better
# Weighted combination:

final_df["performance_score"] = (
      (1 - final_df["riskfactor"]) * 0.5 +
      (final_df["avgmpg"] / final_df["avgmpg"].max()) * 0.3 +
      (1 - (final_df["total_events"] / final_df["total_events"].max())) * 0.2
)

# ----------------------------------------------------
# 7. RANK DRIVERS (Best = rank 1)
# ----------------------------------------------------
final_df["rank"] = final_df["performance_score"].rank(ascending=False).astype(int)

# ----------------------------------------------------
# 8. SORT FOR CLEAN OUTPUT
# ----------------------------------------------------
final_df = final_df.sort_values("rank")

# ----------------------------------------------------
# 9. SAVE FINAL BUSINESS PROBLEM 5 DATASET TO CSV
# ----------------------------------------------------
final_df.to_csv("final_business_problem_5.csv", index=False)

final_df


driverid  riskfactor  total_events    avgmpg  performance_score  rank
12       A2       0.071             1  5.164026           0.916302     1
11      A19       0.000             0  4.148288           0.913749     2
56       A6       0.071             1  5.025321           0.909155     3
33      A39       0.071             1  4.748619           0.894897     4
62      A65       0.071             1  4.742743           0.894594     5
..      ...         ...           ...       ...                ...   ...
95      A95       0.571             8  4.408053           0.527349    96
29      A35       0.643             9  5.266936           0.521319    97
46      A50       0.643             9  4.731050           0.493706    98
71      A73       0.714            10  4.226251           0.417910    99
97      A97       1.000            14  5.562474           0.286618   100

[100 rows x 6 columns]